In [1]:
!pip install transformers datasets
!pip install emot
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 34.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from pythainlp.tokenize import word_tokenize
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pythainlp.corpus.common import thai_stopwords
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
import os
pd.options.mode.chained_assignment = None #for disable warning

In [3]:
dt = load_dataset("wisesight_sentiment")

Generating train split:   0%|          | 0/21628 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2404 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2671 [00:00<?, ? examples/s]

Dataset wisesight_sentiment downloaded and prepared to /root/.cache/huggingface/datasets/wisesight_sentiment/wisesight_sentiment/1.0.0/fc2b1bdfe79571b2e281e4afdb5aac069cf9270bf0f85694239be672a4191969. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train_dt = dt['train']
test_dt = dt['test']
val_dt = dt['validation']

#change dataframe format
train = pd.DataFrame.from_dict(train_dt)
test = pd.DataFrame.from_dict(test_dt)
val = pd.DataFrame.from_dict(val_dt)

In [5]:
text1=train['texts'].values.tolist()
text2=test['texts'].values.tolist()
text3=val['texts'].values.tolist()
print(len(text1))
print(len(text2))
print(len(text3))
text_list = text1+text2+text3
print('total lenght of list',len(text_list))

21628
2671
2404
total lenght of list 26703


In [6]:
sentiment1=train['category'].values.tolist()
sentiment2=test['category'].values.tolist()
sentiment3=val['category'].values.tolist()
print(len(sentiment1))
print(len(sentiment2))
print(len(sentiment3))
sentiment_list = sentiment1+sentiment2+sentiment3
print('total lenght of list',len(sentiment_list))

21628
2671
2404
total lenght of list 26703


In [7]:
data = pd.DataFrame(list(zip(text_list, sentiment_list)),columns =['texts', 'category'])

In [8]:
data

,texts,category
0,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...,1
1,เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...,1
2,บัตรสมาชิกลดได้อีกไหมคับ,1
3,สนใจ new mazda2ครับ,1
4,😍😍,0
...,...,...
26698,ตุณแป้งเห็นน้องที่บริษัทใช้ #snailwhite เลยให้...,1
26699,ราคาก็เท่าๆกันกับ honda หนิครับ เช่น City ก็ระ...,1
26700,ถ้าเท่าบุหรี่มวน พวกมึงก็ควรจับคนพกบุหรีมวนด้ว...,2
26701,ซื้อรถ ford ranger ขับมา9เดือนตอนนี้ยังไม่ได้ป...,2


In [9]:
# 0 = 'positive'
# 1 = 'neutral'
# 2 = 'negative'
# 3 = 'question'

data['category'].value_counts()

1    14539
2     6811
0     4778
3      575
Name: category, dtype: int64

In [10]:
data = data[data['category'] != 3]
data = data[data['category'] != 1]

In [11]:
data['category'] = data['category'].replace(2,1)

In [12]:
data['category'].value_counts()

1    6811
0    4778
Name: category, dtype: int64

In [13]:
#Check null value from dataframe
data.isnull().sum()

texts       0
category    0
dtype: int64

In [14]:
def remove_japanese(x):
    hiragana = re.compile('[\u3040-\u309F]')
    katakana = re.compile('[\u30A0-\u30FF]')
    CJK = re.compile('[\u4300-\u9faf]')

    for i in tqdm(range(len(x))):
        data['tok_texts'].iloc[i] = hiragana.sub('', data['tok_texts'].iloc[i])
        data['tok_texts'].iloc[i] = katakana.sub('', data['tok_texts'].iloc[i])
        data['tok_texts'].iloc[i] = CJK.sub('', data['tok_texts'].iloc[i])

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_stopwords(x):
    thai_stopword = list(thai_stopwords())
    return "".join(token for token in x if token not in thai_stopword)

In [15]:
data['tok_texts'] = data['texts'].apply(lambda x: re.sub(r'http\S+','',x))

data['tok_texts'] = data['tok_texts'].apply(lambda x: re.sub(r'[A-Za-z]+','',x))

data['tok_texts'] = data['tok_texts'].apply(lambda x: re.sub(r'[0-9]+','',x))

data['tok_texts']=data['tok_texts'].str.replace('[^\w\s]','',regex=True)

remove_japanese(data['tok_texts'])

for i in tqdm(range(len(data['tok_texts']))):
    data['tok_texts'].iloc[i] = remove_emoji(data['tok_texts'].iloc[i])

data['tok_texts'].replace('', np.nan, inplace=True)

data.isnull().sum()
data.dropna(inplace = True)

data['tok_texts'] = data['tok_texts'].apply(lambda x: ' '.join(word_tokenize(x)))

for i in tqdm(range(len(data['tok_texts']))):
    data['tok_texts'].iloc[i] = remove_stopwords(data['tok_texts'].iloc[i])

data['tok_texts'] = data['tok_texts'].apply(lambda x: re.sub(" +"," ",x))

#Drop columns that are not required
data.drop(columns=['texts'],axis=0,inplace=True)

100%|██████████| 11531/11531 [00:15<00:00, 765.30it/s]


In [16]:
data

,category,tok_texts
5,0,พอ กบ รถ อะไร วะ ทรง ประหลาด สวย ลำ คม
6,1,ออ ออ ผา อ นาม ย แบบ สอด อะนะ กก ลว มาก
8,1,ไม ขอบ ส โทน น แล ว ไป ชอบ โทน หย กด ย มาก กก
9,1,ไป จบ บหร ไฟ ฟา ใน คล ป ตำรวจ เปนคน ได ละ มง ผ...
10,0,เหมาะ กะมง ทสส คะ เอย สด
...,...,...
26692,1,อก เส ยง คะ ขาว ผด กระ เท ยม ไม หอม ไมอ รอย ขา...
26693,0,ชอบ ขอ ตง ผหญง แดก อยาก แดก ไมมตง จา ย ให ผญ จ...
26697,0,ส ครบ ผม เปนคนหนง ทสบ ไฟ ฟา เป นอก ท เส ยง ครบ
26700,1,ถา เทา บห รม วน พวก มง กค วร จบ คน พก บห รม วน...


In [17]:
data['category'] = data['category'].replace(0,'Positive')
data['category'] = data['category'].replace(1,'Negative')

In [18]:
data

,category,tok_texts
5,Positive,พอ กบ รถ อะไร วะ ทรง ประหลาด สวย ลำ คม
6,Negative,ออ ออ ผา อ นาม ย แบบ สอด อะนะ กก ลว มาก
8,Negative,ไม ขอบ ส โทน น แล ว ไป ชอบ โทน หย กด ย มาก กก
9,Negative,ไป จบ บหร ไฟ ฟา ใน คล ป ตำรวจ เปนคน ได ละ มง ผ...
10,Positive,เหมาะ กะมง ทสส คะ เอย สด
...,...,...
26692,Negative,อก เส ยง คะ ขาว ผด กระ เท ยม ไม หอม ไมอ รอย ขา...
26693,Positive,ชอบ ขอ ตง ผหญง แดก อยาก แดก ไมมตง จา ย ให ผญ จ...
26697,Positive,ส ครบ ผม เปนคนหนง ทสบ ไฟ ฟา เป นอก ท เส ยง ครบ
26700,Negative,ถา เทา บห รม วน พวก มง กค วร จบ คน พก บห รม วน...


In [19]:
train = list(data["tok_texts"])
answer = list(data["category"])

In [20]:
X_train, X_val, y_train, y_val = train_test_split(train, answer, test_size=0.2,stratify=answer)

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
from transformers import BertTokenizer,BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Sentiment Analysis Thai Language/Pytorch Model")

In [24]:
model = model.to('cuda')

In [34]:
def predict(text):
  # data = pd.DataFrame(text)
  inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt',max_length=512).to('cuda')
  outputs = model(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  if(np.argmax(predictions) == 0):
    return "Positive"
  elif (np.argmax(predictions) == 1):
    return "Negative"

In [38]:
predict_label = []
for sentence in tqdm(X_val):
  predict_label.append(predict(sentence))

100%|██████████| 2307/2307 [00:43<00:00, 53.60it/s]


In [41]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_val, predict_label))

print(confusion_matrix(y_val, predict_label))

              precision    recall  f1-score   support

    Negative       0.96      0.97      0.97      1361
    Positive       0.96      0.95      0.95       946

    accuracy                           0.96      2307
   macro avg       0.96      0.96      0.96      2307
weighted avg       0.96      0.96      0.96      2307

[[1319   42]
 [  51  895]]


In [45]:
print('Accuracy Score',accuracy_score(y_val, predict_label)*100,'%')
print('Precision Macro Score',precision_score(y_val, predict_label,average = 'macro')*100,'%')
print('Recall_Score',recall_score(y_val, predict_label, average = 'macro')*100,'%')
print('F1_Score',f1_score(y_val, predict_label, average = 'macro')*100,'%')

Accuracy Score 95.96879063719116 %
Precision Macro Score 95.89749082722464 %
Recall_Score 95.76145664563893 %
F1_Score 95.82786336457187 %
